# Keyphrases in Plot Summaries

## Phrase mining and filtering

(TODO : Definition of a high quality phrase)

(TODO : How this will help us understand the interaction of technology and cinema)

In [46]:
# Imports
import json
import pandas as pd
from helpers import print_versions, load_data_with_columns

print_versions([pd, json])

pandas==2.1.1
json==2.0.9


In [47]:
# Loading data
DATAFOLDER = '../data/'
plot_summaries = load_data_with_columns(DATAFOLDER, 'plot_summaries.txt')
movie_metadata = load_data_with_columns(DATAFOLDER, 'movie.metadata.tsv')

# Exploratory Data Analysis

In [48]:
# Display movie metadata
movie_metadata

,movie_wikipedia_id,movie_freebase_id,movie_name,movie_release_date,box_office,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [49]:
# Display plot summaries
plot_summaries

,movie_wikipedia_id,plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."


## Missing or duplicate data

There appear to be more rows in movie_metadata than in plot_summaries. We diagnose the problem.

1. We check for duplicates : 
    + Are the wikipedia IDs unique in movie_metadata and plot_summaries?
        + Answer: Yes.
    + Are there duplicate summary texts?
        + Answer: Yes.
2. We check for missing data :
    + Are the wikipedia IDs in plot_summaries a subset of those in movie_metadata?
        + Answer: No, there are 99 summaries with no associated metadata.
    + What percentage of movies are missing per year?
        + Answer: ???

In [51]:
# The wikipedia_id's are unique in both tables.
assert(movie_metadata['movie_wikipedia_id'].is_unique)
assert(plot_summaries['movie_wikipedia_id'].is_unique)

# Summaries with no metadata
movie_metadata_ids = movie_metadata['movie_wikipedia_id'].to_list()
plot_summaries_ids = plot_summaries['movie_wikipedia_id'].to_list()
plot_no_metadata = [id for id in plot_summaries_ids if id not in movie_metadata_ids]

In [80]:
# Number of summaries with no metadata.
print("There are {:d} plot summaries with no associated metadata".format(len(plot_no_metadata)))
plot_summaries.set_index('movie_wikipedia_id').loc[plot_no_metadata].reset_index()

There are 99 plot summaries with no associated metadata


,movie_wikipedia_id,plot
0,2862137,Sathyanarayan Murthy is a rough but kind man ...
1,33334420,{{more plot}} Mural tells the story of Zhu Xia...
2,16758721,"The story is about a young man “Tony”, a 25 ye..."
3,23217064,"Years before the events of the first film, Lo..."
4,2746943,"A young 16-year-old girl, Zhu Ying Tai, manage..."
...,...,...
94,35607003,Parisian Marion is living in New York with her...
95,32942181,"Rajat , Nishant ""Liquid"" Agarwal and Vikrant ..."
96,133671,Gerry Conlon is shown in Belfast stripping le...
97,34076714,"After a hit gone wrong, Nikita, an exiled Russ..."


In [136]:
# Duplicate plot texts
duplicated_plots = plot_summaries[plot_summaries['plot'].duplicated(keep=False)]
print("There are {:d} plot summaries that appear more than once.".format(len(duplicated_plots['plot'].unique())))

There are 5 plot summaries that appear more than once.


In [128]:
# Left-merge movie metadata with plots. This disregards the 99 plots with no metadata
movies_with_plots = movie_metadata.merge(plot_summaries, how='left')

# There are movies different titles but identical plots.
movies_with_plots.set_index('movie_wikipedia_id').loc[duplicated_plots['movie_wikipedia_id']].groupby('plot')['movie_name'].unique().reset_index()

,plot,movie_name
0,A woman is thrown out of her home by her jealo...,"[Madame X, The Trial of Madame X]"
1,An orphan Raghav turns into a ruthless contrac...,"[Antham, Drohi]"
2,"As the American Civil War begins, Ned Burton l...",[The Warrens of Virginia]
3,Raja was adopted by a criminal don at a very ...,[Amar Deep]
4,Taking a step aside from the usual serious dra...,"[Andhamaina Anubhavam, Ninaithale Inikkum]"


What can we know about these movies whose plots appear multiple times?

In [141]:
# Print data about those movies who share a plot with another movie.
movie_metadata[movie_metadata['movie_wikipedia_id'].isin(duplicated_plots['movie_wikipedia_id'])][['movie_name', 'movie_release_date', 'languages', 'countries']].sort_values(by='movie_name')

,movie_name,movie_release_date,languages,countries
22185,Amar Deep,1979,"{""/m/03k50"": ""Hindi Language""}","{""/m/03rk0"": ""India""}"
32255,Amar Deep,NaN,"{""/m/03k50"": ""Hindi Language""}",{}
59148,Andhamaina Anubhavam,1979,"{""/m/09s02"": ""Telugu language""}","{""/m/03rk0"": ""India""}"
67464,Antham,1992-09-11,"{""/m/09s02"": ""Telugu language""}","{""/m/03rk0"": ""India""}"
28621,Drohi,1992-10-25,"{""/m/03k50"": ""Hindi Language""}","{""/m/03rk0"": ""India""}"
18993,Madame X,1916-01-14,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}"
49381,Madame X,1920-09-26,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}"
57569,Madame X,1954,"{""/m/0349s"": ""Greek Language""}","{""/m/035qy"": ""Greece""}"
65014,Madame X,1981-03-16,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}"
59142,Ninaithale Inikkum,1979-04-14,"{""/m/07c9s"": ""Tamil Language""}","{""/m/03rk0"": ""India""}"


We note the following:
1. *Andhamaina Anubhavam* / *Ninaithale Inikkum* are the Telugu and Tamil names for the same movie. 
2. *Antham* / *Drohi* are the Telugu and Hindi names for the same movie.
3. There are multiple movie adaptations of the play *(The Trial of) Madame X*. 
4. There is a "lost" 1924 version of the movie The Warrens of Virginia, with the same plot, but it appears in this dataset.
5. The movie *Amar Deep* appearing twice appears to simply be a dataset error.

What is the percentage of missing plot summary data per year?